In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import dlib
import glob
import PIL
import os
import cv2
from PIL import Image
from tqdm import tqdm
import numpy as np
from os import path

In [0]:
import os,cv2
# import a classifier object
from sklearn.svm import LinearSVC
from numpy import zeros, newaxis
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
from __future__ import print_function
import torch
#import PIL
#print(PIL.PILLOW_VERSION)
import glob
import socket
import csv
import pandas as pd
import keras
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense

#from tflearn.layers.conv import conv_2d, max_pool_2d
from keras.layers import Dropout

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
from keras.preprocessing import image

import tensorflow as tf
import keras
from keras.utils import np_utils
from keras import backend as K
from keras import callbacks
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#ada
from keras.optimizers import SGD,Adadelta

#network
from keras import backend as K

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential

from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)

In [0]:
def load_img(path):
    img = cv2.imread(path)
    return img


def draw_predict(frame, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)


def get_eyes_nose_dlib(shape):
    nose = shape[4][1]
    left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
    left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
    right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
    right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
    return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)


def get_eyes_nose(eyes, nose):
    left_eye_x = int(eyes[0][0] + eyes[0][2] / 2)
    left_eye_y = int(eyes[0][1] + eyes[0][3] / 2)
    right_eye_x = int(eyes[1][0] + eyes[1][2] / 2)
    right_eye_y = int(eyes[1][1] + eyes[1][3] / 2)
    nose_x = int(nose[0][0] + nose[0][2] / 2)
    nose_y = int(nose[0][1] + nose[0][3] / 2)

    return (nose_x, nose_y), (right_eye_x, right_eye_y), (left_eye_x, left_eye_y)


def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy


def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False


def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a


def show_img(img):
    while True:
        cv2.imshow('face_alignment_app', img)
        c = cv2.waitKey(1)
        if c == 27:
            break
    cv2.destroyAllWindows()


def shape_to_normal(shape):
    shape_normal = []
    for i in range(0, 5):
        shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
    return shape_normal


def rotate_opencv(img, nose_center, angle):
    M = cv2.getRotationMatrix2D(nose_center, angle, 1)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC)
    return rotated


def rotation_detection_dlib(img, mode, show=False):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/gdrive/My Drive/Machine learing/Pre-process/Facial corection/shape_predictor_5_face_landmarks.dat')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    if len(rects) > 0:
        for rect in rects:
            x = rect.left()
            y = rect.top()
            w = rect.right()
            h = rect.bottom()
            shape = predictor(gray, rect)
            shape = shape_to_normal(shape)
            nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
            center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
            center_pred = (int((x + w) / 2), int((y + y) / 2))
            length_line1 = distance(center_of_forehead, nose)
            length_line2 = distance(center_pred, nose)
            length_line3 = distance(center_pred, center_of_forehead)
            cos_a = cosine_formula(length_line1, length_line2, length_line3)
            angle = np.arccos(cos_a)
            rotated_point = rotate_point(nose, center_of_forehead, angle)
            rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
            if is_between(nose, center_of_forehead, center_pred, rotated_point):
                angle = np.degrees(-angle)
            else:
                angle = np.degrees(angle)

            if mode:
                img = rotate_opencv(img, nose, angle)
            else:
                img = Image.fromarray(img)
                img = np.array(img.rotate(angle))
        if show:
            show_img(img)
        return img
    else:
        return img


def rotation_detection_opencv(img, mode, show=False):
    nose_cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/Machine learing/Pre-process/Facial corection/haarcascade_mcs_nose.xml')
    eyes_cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/Machine learing/Pre-process/Facial corection/haarcascade_eye.xml')
    fase_cascade = cv2.CascadeClassifier('/content/gdrive/My Drive/Machine learing/Pre-process/Facial corection/haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    nose_rects = nose_cascade.detectMultiScale(gray, 1.3, 5)
    eyes_rects = eyes_cascade.detectMultiScale(gray, 1.3, 5)
    face_rects = fase_cascade.detectMultiScale(gray, 1.3, 5)
    length_eyes = len(eyes_rects)

    if length_eyes == 2 and len(nose_rects) != 0 and len(face_rects) != 0:
        nose, right_eye, left_eye = get_eyes_nose(eyes_rects, nose_rects)
    else:
        print("Couldn't determine eyes/nose")
        return img
    center_of_forehead = (int((right_eye[0] + left_eye[0]) / 2), int((right_eye[1] + left_eye[1]) / 2))
    center_pred = (int((face_rects[0][0] + face_rects[0][2]) / 2), int((face_rects[0][1] + face_rects[0][1]) / 2))
    length_line1 = distance(center_of_forehead, nose)
    length_line2 = distance(center_pred, nose)
    length_line3 = distance(center_pred, center_of_forehead)
    cos_a = cosine_formula(length_line1, length_line2, length_line3)
    angle = np.arccos(cos_a)
    rotated_point = rotate_point(nose, center_of_forehead, angle)
    rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
    if is_between(nose, center_of_forehead, center_pred, rotated_point):
        angle = np.degrees(-angle)
    else:
        angle = np.degrees(angle)
    if mode:
        img = rotate_opencv(img, nose, angle)
    else:
        img = Image.fromarray(img)
        img = np.array(img.rotate(angle))
    if show:
        show_img(img)
    return img


def save_img(path, img):
    cv2.imwrite(path, img)

In [0]:
Pre_process ="/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate"
data_path = '/content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/CK+48'
data_dir_list = os.listdir(data_path)
img_data_list=[]

for dataset in data_dir_list:    
  img_list=os.listdir(data_path+'/'+ dataset)
  if not os.path.exists(Pre_process +'/'+ dataset):
    os.mkdir(Pre_process +'/'+ dataset)
    print(" Created "+ dataset)
  else:
    print(Pre_process + dataset) 
    print ('Load dataset-'+'{}\n'.format(dataset))
  for img in img_list:
    path_to_load =data_path + '/'+ dataset + '/'+ img 
    print("data load: ", path_to_load)
    path_to_save = Pre_process + '/'+ dataset + '/'+ img
    print("data_save ",path_to_save)               
    img = load_img(path_to_load)
    img=cv2.resize(img,(256,256))
    img = rotation_detection_dlib(img, True)
    save_img(path_to_save, img)

/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotatecontempt
Load dataset-contempt

data load:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/CK+48/contempt/S158_002_00000009.png
data_save  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate/contempt/S158_002_00000009.png
data load:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/CK+48/contempt/S149_002_00000012.png
data_save  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate/contempt/S149_002_00000012.png
data load:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/CK+48/contempt/S138_008_00000009.png
data_save  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate/contempt/S138_008_00000009.png
data load:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/CK+48/contempt/S156_002_00000019.png
data_save  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate/contempt/S156_002_00000019.png
data load:  /cont

In [0]:
#2.Crop FACE
#crop face step
## DIRECTORY of the images
data_directory = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+_rotate"
result_directory = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Crop img full"
#data_path_cr = '/content/gdrive/My Drive/Machine learing/Test/Final Pr_3/result'
data_dir_list = os.listdir(data_directory)

## directory where the images to be saved:
#f_directory = "/content/gdrive/My Drive/ML Backup/jaffe"
def facecrop(image_path, result_img_path):
    ## Crops the face of a person from any image!

    ## OpenCV XML FILE for Frontal Facial Detection using HAAR CASCADES.
    face_model = "/content/gdrive/My Drive/ML Backup/haarcascade_frontalface_alt.xml"
    cascade = cv2.CascadeClassifier(face_model)
    #cv2.CascadeClassifier.load(facedata)
    ## Reading the given Image with OpenCV
    img = cv2.imread(image_path)

    try:
        ## Some downloaded images are of unsupported type and should be ignored while raising Exception, so for that
        ## I'm using the try/except functions.

        faces = cascade.detectMultiScale(img)
        for (x,y,w,h) in faces:
            
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            sub_face = img[y:y+h, x:x+w]
            cv2.imwrite(result_img_path, sub_face)

            print("writing: ", result_img_path)
            #f_name = image.split('/')
    except Exception as e:
        print("exception: ", e)
if __name__ == '__main__':
    for dataset in data_dir_list:
        print("emotion: ", dataset)
        emotion_folder = os.path.join(data_directory, dataset)
        result_emotion_folder = os.path.join(result_directory, dataset)
        if not os.path.isdir(result_emotion_folder):
          os.mkdir(result_emotion_folder)
          print("Creating folder: ", result_emotion_folder)
        img_list=next(os.walk(emotion_folder))[2]     
        for img_name in img_list:
          file_path = os.path.join(emotion_folder, img_name)
          result_path = os.path.join(result_emotion_folder, img_name)
          print("processing: ", file_path)
          facecrop(file_path, result_path)

In [0]:
#3. Illu
import logging
import numpy as np

# Homomorphic filter class
class HomomorphicFilter:
    def __init__(self, a = 0.5, b = 1.5):
        self.a = float(a)
        self.b = float(b)

    # Filters
    def __butterworth_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
        return (1 - H)

    def __gaussian_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        H = np.zeros(I_shape)
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = np.exp((-Duv/(2*(filter_params[0])**2)))
        return (1 - H)

    # Methods
    def __apply_filter(self, I, H):
        H = np.fft.fftshift(H)
        I_filtered = (self.a + self.b*H)*I
        return I_filtered

    def filter(self, I, filter_params, filter='butterworth', H = None):
        #  Validating image
        if len(I.shape) is not 2:
            raise Exception('Improper image')

        # Take the image to log domain and then to frequency domain 
        I_log = np.log1p(np.array(I, dtype="float"))
        I_fft = np.fft.fft2(I_log)

        # Filters
        if filter=='butterworth':
            H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='gaussian':
            H = self.__gaussian_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='external':
            print('external')
            if len(H.shape) is not 2:
                raise Exception('Invalid external filter')
        else:
            raise Exception('Selected filter not implemented')
        
        # Apply filter on frequency domain then take the image back to spatial domain
        I_fft_filt = self.__apply_filter(I = I_fft, H = H)
        I_filt = np.fft.ifft2(I_fft_filt)
        I = np.exp(np.real(I_filt))-1
        return np.uint8(I)
# End of class HomomorphicFilter
if __name__ == "__main__":
  img_illu_out = []
  fl_illu_imgout = []
  img_data_list=[]
  n=0
  illu_in = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Crop img full"
  fl_illu_in = os.listdir(illu_in)
  fl_origin_illu = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+full"
  img_crop = []
  

  
  for fd_crop in fl_illu_in:
    if not os.path.isdir(fl_origin_illu + '/'+ fd_crop):
      fl_illu_imgout = fl_origin_illu + '/'+ fd_crop
      os.mkdir(fl_illu_imgout)
      print("Creating" + fd_crop )
    crop_list = os.listdir(illu_in + '/' + fd_crop)
    for crop_img in crop_list:
      n=n+1
      img_path_in = illu_in + '/'+ fd_crop + '/'+crop_img
      #img_illu_out = os.path.join(fl_illu_imgout , crop_img)
      img_path_out  = fl_origin_illu+'/' + fd_crop+ '/'+ crop_img

      img = cv2.imread(img_path_in)[:, :, 0]
      homo_filter = HomomorphicFilter(a = 0.75, b = 1.25)
      img_filtered = homo_filter.filter(I=img, filter_params=[30,2])
      cv2.imwrite(img_path_out, img_filtered)
      print("Done illuminate ", crop_img)

Creatingdisgust
Done illuminate  S052_006_00000011.png
Done illuminate  S132_005_00000015.png
Done illuminate  S080_008_00000008.png
Done illuminate  S088_004_00000019.png
Done illuminate  S080_008_00000009.png
Done illuminate  S046_004_00000016.png
Done illuminate  S075_005_00000012.png
Done illuminate  S022_006_00000017.png
Done illuminate  S102_009_00000014.png
Done illuminate  S097_004_00000030.png
Done illuminate  S044_006_00000018.png
Done illuminate  S060_005_00000019.png
Done illuminate  S096_003_00000010.png
Done illuminate  S097_004_00000028.png
Done illuminate  S107_005_00000009.png
Done illuminate  S044_006_00000019.png
Done illuminate  S088_004_00000020.png
Done illuminate  S080_008_00000007.png
Done illuminate  S055_003_00000008.png
Done illuminate  S061_004_00000021.png
Done illuminate  S085_004_00000015.png
Done illuminate  S076_005_00000012.png
Done illuminate  S035_005_00000019.png
Done illuminate  S077_006_00000012.png
Done illuminate  S056_002_00000008.png
Done illu

In [0]:
#4.rename images
#CK+
data_path="/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+"
data_dir_list = os.listdir(data_path)
img_data_list=[]
for dataset in data_dir_list:
  x=0
  data= os.listdir(data_path+"/"+dataset)
  for img in data:
    print("file remane: ",data_path+"/"+dataset+"/"+img )
    source_link= data_path+"/"+dataset+"/"+img
    os.rename(source_link ,data_path+"/"+ dataset+"/"+dataset +str(x)+".png")
    x +=1
    print("done ",data_path+"/"+ dataset+"/"+dataset +str(x-1)+".png")

file remane:  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/S074_001_00000018.png
done  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/fear0.png
file remane:  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/S050_001_00000017.png
done  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/fear1.png
file remane:  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/S132_003_00000023.png
done  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/fear2.png
file remane:  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/S138_001_00000011.png
done  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/fear3.png
file remane:  /content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+/fear/S032

In [0]:
#JAFFE
data_path="/content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2"
data_dir_list = os.listdir(data_path)
img_data_list=[]
for dataset in data_dir_list:
  x=0
  data= os.listdir(data_path+"/"+dataset)
  for img in data:
    print("file remane: ",data_path+"/"+dataset+"/"+img )
    source_link= data_path+"/"+dataset+"/"+img
    os.rename(source_link ,data_path+"/"+ dataset+"/"+dataset +str(x)+".png")
    x +=1
    print("done ",data_path+"/"+ dataset+"/"+dataset +str(x-1)+".png")

file remane:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happy14.jpg
done  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happiness0.png
file remane:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happy1.jpg
done  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happiness1.png
file remane:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happy11.jpg
done  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happiness2.png
file remane:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happy12.jpg
done  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happiness3.png
file remane:  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/happiness/happy10.jpg
done  /content/gdrive/My Drive/Machine learing/Test/Final Pr_3/Data/JAFFE2/h

In [0]:
#5. Data argument
#mirror img
data_path = '/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+'
data_dir_list = os.listdir(data_path)
img_data_list=[]
emotion =[]
emotion =["anger", "contempt", "disgust", "fear", "happy", "neutral","sadness", "surprise"]
path_after_flip = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+ After mirror"
i = 0
n = [0,0,0,0,0,0,0,0]

for dataset in data_dir_list:
  #anger (+116) =250
    if (dataset == emotion[0]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[0] < 116):
          n[0] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[0])
        else:
          print(" Chi den anh 116 trong ", dataset)
          break
  #contempt 53 -> x2 = 106
    if (dataset == emotion[1]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[1] < 53):
          n[1] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[1])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #disgust 176 -> +73 =249
    if (dataset == emotion[2]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[2] < 73):
          n[2] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[2])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #fear 74 -> x2 = 148
    if (dataset == emotion[3]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[3] < 74):
          n[3] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[3])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #happy 206 + 40= 246
    if (dataset == emotion[4]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[4] < 40):
          n[4] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[4])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #neutral 180 + 60= 240
    if (dataset == emotion[5]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[5] < 60):
          n[5] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[5])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #sadness 83 +77 = 160
    if (dataset == emotion[6]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[6] < 77):
          n[6] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          iim_mirror = ImageOps.mirror(im)
          iim_mirror.save(path_after_flip+"/"+dataset+"/mirror"+img)
          print("Done flipping: mirror", img)

          print("anh so ", n[6])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #surprise 248
print(" n[] la: ",n)

In [0]:
#Filp img 90
data_path = '/content/gdrive/My Drive/Machine learing/Pre-process/CK+/Data_af_prepocessingCK+'
data_dir_list = os.listdir(data_path)
img_data_list=[]
emotion =[]
emotion =["anger", "contempt", "disgust", "fear", "happy", "neutral","sadness", "surprise"]
path_after_flip = "/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+ After flip"
i = 0
n = [0,0,0,0,0,0,0,0]
#contempt fear sadness 1, 3, 6
for dataset in data_dir_list:
  #contempt  106 +100
    if (dataset == emotion[1]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[1] < 100):
          n[1] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          im_flip = ImageOps.flip(im)
          im_flip.save(path_after_flip+"/"+dataset+"/flip"+img)
          print("Done flipping: flip", img)

          print("anh so ", n[1])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #fear  148+100
    if (dataset == emotion[3]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[3] < 100):
          n[3] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          im_flip = ImageOps.flip(im)
          im_flip.save(path_after_flip+"/"+dataset+"/flip"+img)
          print("Done flipping: flip", img)

          print("anh so ", n[3])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #sadness 160 +90
    if (dataset == emotion[6]):
      img_list=os.listdir(data_path+'/'+ dataset)
      print ('Load dataset-'+'{}\n'.format(dataset))
      for img in img_list:
        if(n[6] < 90):
          n[6] +=1
          im = Image.open(data_path + "/"+dataset + "/"+ img)
          im_flip = ImageOps.flip(im)
          im_flip.save(path_after_flip+"/"+dataset+"/flip"+img)
          print("Done flipping: flip", img)

          print("anh so ", n[6])
        else:
          print(" Chi den anh  trong ", dataset)
          break
  #surprise 248
print(" n[] la: ",n)

In [0]:
#Gauss
import cv2
from skimage.util import *
data_path="/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+ After mirror/contempt"
path_save_gauss ="/content/gdrive/My Drive/Machine learing/Pre-process/CK+/CK+ After flip/contempt_after_gaussing"
k=0
data_dir = os.listdir(data_path)
for data in data_dir:
  if k < 50:
    im = Image.open(data_path + "/"+data)
    # convert PIL Image to ndarray
    im_arr = np.asarray(im)

    # random_noise() method will convert image in [0, 255] to [0, 1.0],
    # inherently it use np.random.normal() to create normal distribution
    # and adds the generated noised back to image
    noise_img = random_noise(im_arr, var=0.05**2)
    noise_img = (255*noise_img).astype(np.uint8)

    img = Image.fromarray(noise_img)
    img.save(path_save_gauss + "/gaussed"+data)
    print(" Gaussed ",data)
    k+=1
  else:
    print(" Gaussed 50 img")
    break